In [25]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import PIL
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')
import tensorflow_addons as tfa
from tensorflow.keras.metrics import top_k_categorical_accuracy, categorical_accuracy

In [26]:
root = "D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Complete Dataset"

In [27]:
train_dir = root+'/Hair Removed Images/train'
val_dir = root+'/Hair Removed Images/val'
test_dir = root+'/Hair Removed Images/test'
datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

# test_datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=8,class_mode='categorical',seed=42)

val_data = datagen.flow_from_directory(val_dir,target_size=(224,224),batch_size=8,class_mode='categorical',seed=42)

test_data = datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=8,class_mode='categorical',shuffle=False)

Found 40253 images belonging to 7 classes.
Found 1000 images belonging to 7 classes.
Found 1006 images belonging to 7 classes.


In [33]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

num_labels = 7

base_model = InceptionV3(include_top=True, pooling = 'max', weights = 'imagenet',classes=1000)
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation = 'softmax',))

96124928/96112376 [==============================] - 100s 1us/step


In [34]:
def top_2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

lrReduction=ReduceLROnPlateau(monitor='val_accuracy',
                              factor=0.5,
                              patience=2,
                              verbose=1,
                              mode='auto',
                              min_lr=0.00001)

early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, verbose=1)

In [35]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',
                                                                        top_2_acc,
                                                                        top_3_acc,
                                                                        tf.metrics.AUC(curve='ROC'),
                                                                        tfa.metrics.MatthewsCorrelationCoefficient(num_classes=7)])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 1000)              23851784  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               128128    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 903       
Total params: 23,980,815
Trainable params: 23,946,383
Non-trainable params: 34,432
_________________________________________________________________


In [36]:
step_size_train=train_data.n//train_data.batch_size
history = model.fit_generator(generator=train_data,
                   steps_per_epoch=step_size_train,
                   validation_data=val_data,
                    epochs=50, 
                    verbose = 1,
                    callbacks=[lrReduction,early_stopping])

Epoch 1/50
5031/5031 [==============================] - 648s 128ms/step - loss: 1.8991 - accuracy: 0.1881 - top_2_acc: 0.3476 - top_3_acc: 0.5014 - auc_3: 0.5773 - MatthewsCorrelationCoefficient: 0.0517 - val_loss: 1.6789 - val_accuracy: 0.5370 - val_top_2_acc: 0.6440 - val_top_3_acc: 0.7560 - val_auc_3: 0.7944 - val_MatthewsCorrelationCoefficient: 0.0856
Epoch 2/50
5031/5031 [==============================] - 640s 127ms/step - loss: 1.7556 - accuracy: 0.2826 - top_2_acc: 0.4748 - top_3_acc: 0.6404 - auc_3: 0.6888 - MatthewsCorrelationCoefficient: 0.1709 - val_loss: 1.5012 - val_accuracy: 0.5790 - val_top_2_acc: 0.6830 - val_top_3_acc: 0.7930 - val_auc_3: 0.8249 - val_MatthewsCorrelationCoefficient: 0.1494
Epoch 3/50
5031/5031 [==============================] - 639s 127ms/step - loss: 1.6595 - accuracy: 0.3300 - top_2_acc: 0.5448 - top_3_acc: 0.7203 - auc_3: 0.7406 - MatthewsCorrelationCoefficient: 0.2249 - val_loss: 1.2587 - val_accuracy: 0.5730 - val_top_2_acc: 0.6790 - val_top_3_acc

In [37]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(train_data)

5032/5032 [==============================] - 200s 40ms/step - loss: 0.9624 - accuracy: 0.6195 - top_2_acc: 0.8036 - top_3_acc: 0.9044 - auc_3: 0.9220 - MatthewsCorrelationCoefficient: 0.5578


In [38]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(val_data)

125/125 [==============================] - 5s 40ms/step - loss: 0.8302 - accuracy: 0.6630 - top_2_acc: 0.8100 - top_3_acc: 0.9480 - auc_3: 0.9408 - MatthewsCorrelationCoefficient: 0.3625


In [39]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(test_data)

126/126 [==============================] - 10s 81ms/step - loss: 0.8977 - accuracy: 0.6412 - top_2_acc: 0.7734 - top_3_acc: 0.9245 - auc_3: 0.9304 - MatthewsCorrelationCoefficient: 0.3251


In [40]:
from sklearn import metrics

true_test_labels=test_data.labels

true_test_labels=np.array(true_test_labels)

np.unique(true_test_labels,return_counts=True)

pred_test_labels=model.predict(test_data)

pred_test_labels=np.argmax(pred_test_labels,axis=1)

metrics.accuracy_score(true_test_labels,pred_test_labels)

report = metrics.classification_report(true_test_labels,pred_test_labels)
print(report)

              precision    recall  f1-score   support

           0       0.38      0.09      0.14        34
           1       0.33      0.52      0.40        52
           2       0.25      0.15      0.19       111
           3       0.20      0.08      0.12        12
           4       0.26      0.38      0.31       112
           5       0.82      0.81      0.82       670
           6       0.59      0.67      0.62        15

    accuracy                           0.64      1006
   macro avg       0.40      0.39      0.37      1006
weighted avg       0.64      0.64      0.64      1006



In [41]:
# model.save("D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Saved Models\Inception_V3.h5")